<a href="https://colab.research.google.com/github/andresgd17/MLProject_HandLearRecog/blob/main/project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project 3

We load the mfeat-pix.txt data file

In [1]:
import pandas as pd
import numpy  as np
from numpy.linalg import inv
from scipy.linalg import block_diag


mfeat_pix = pd.read_csv('mfeat-pix.txt',sep='\s+',header=None)
mfeat_pix = pd.DataFrame(mfeat_pix)
rows = 15
cols = 16

In [2]:
pickIndices_trainFinal = [j+200*i for i in range(0, 10, 1) for j in range(0, 180, 1)]
pickIndices_testFinal = [j - 20 + 200*(i+1) for i in range(0, 10, 1) for j in range(0, 20, 1)]

#print(pickIndices_trainFinal)
#print(pickIndices_testFinal)
meanTrainImagesFinal = [([0]*10) for i in range(240)]

trainPatternsFinal = []
testPatternsFinal  = []

for i in pickIndices_trainFinal:
    temp_trainFinal = []
    for j in range(rows*cols):
        temp_trainFinal.append(mfeat_pix[j][i])
        #temp_testFinal.append(mfeat_pix[j][i+100])
    trainPatternsFinal.append(temp_trainFinal)
    #testPatternsFinal.append(temp_testFinal)

for i in pickIndices_testFinal:
    temp_testFinal = []
    for j in range(rows*cols):
        #temp_trainFinal.append(mfeat_pix[j][i])
        temp_testFinal.append(mfeat_pix[j][i])
    #trainPatternsFinal.append(temp_trainFinal)
    testPatternsFinal.append(temp_testFinal)


and define the function to split data into training and test data

In [3]:
def cross_validation_split(trainPatternsFinal):
  #split to 160 training and 20 testing data for cross validation
  pickIndices_train = [j+180*i for i in range(0, 10, 1) for j in range(0, 180, 1)]
  pickIndices_test = []
  number_of_tests = 20
  number_of_digits = 10
  for i in range(number_of_digits):
    j = 0
    while j < number_of_tests:
      tmp_pick = 180*i + int(np.random.uniform()*180)
      if tmp_pick in pickIndices_test:
        j -= 1
      else:
        pickIndices_test.append(tmp_pick) 
      j += 1
  pickIndices_test.sort()
  for obj in pickIndices_test:
    pickIndices_train.remove(obj)
  #print(pickIndices_train)
  #print(pickIndices_test)
  #meanTrainImages = [([0]*10) for i in range(240)]

  trainPatterns = []
  testPatterns  = []

  for i in pickIndices_train:
      temp_train = []
      #print(i)
      for j in range(rows*cols):
          temp_train.append(trainPatternsFinal[i][j])  
      trainPatterns.append(temp_train)

  for i in pickIndices_test:
      temp_test = []
      for j in range(rows*cols):
          temp_test.append(trainPatternsFinal[i][j])
      testPatterns.append(temp_test)
  
  meanTrainImages = [] #240x10

  for i in range(0, 240):
    temp = []
    for j in range(0,10):
        mean = 0
        for k in range(j*160,(j+1)*160):
            mean += trainPatterns[k][i]
        #print(i,j*100,(j+1)*100, "mean: ", mean) 
        temp.append(mean/160)
    meanTrainImages.append(temp)

  #pd.DataFrame(meanTrainImages)

  #pd.DataFrame(trainPatterns)
  ##pd.DataFrame(pickIndices_train)
  #pd.DataFrame(pickIndices_test)
  return trainPatterns, testPatterns, meanTrainImages

We define three more functions :


*   training function
*   best model
*   nrOfMisclassificationsTrain



In [4]:
def nrOfMisclassificationsTrain(A,B):
    sum = 0
    for i in range(len(A[0])):
        if (A[0][i] != B[0][i]):
            sum += 1
    return sum
    
def trainingFunction(trainPatterns, testPatterns, meanTrainImages):
  ###################################################################################
  ### defining the trainlabels   160 IS THE NUMBER OF DATA
  b = np.ones(160)
  trainLabels = block_diag(*([b] * 10))
  testLabels  = trainLabels

  ### create a row vector of correct class labels (from 1 ... 10 for the 10 classes). 
  ### This vector is the not the same for training and testing.  
  ### 160 IS FOR THE TRAINING 
  ###  20 IS FOR THE TESTING
  ### Corrrelabels for training data
  correctLabelsTrain, tmpTrain = [], []
  for i in range(0, 10):
      for j in range(i*160, (1+i)*160):
          tmpTrain.append(i+1)
  correctLabelsTrain.append(tmpTrain)

  ### Correct labels for test data
  correctLabelsTest, tmpTest = [], []
  for i in range(0, 10):
      for j in range(i*20, (1+i)*20):
          tmpTest.append(i+1)
  correctLabelsTest.append(tmpTest)
  ##################################################################################

  ### Defining the X matrix given by featureValuesTrain
  featureValuesTrain = np.dot(np.transpose(meanTrainImages), np.transpose(trainPatterns))
  featureValuesTest  = np.dot(np.transpose(meanTrainImages), np.transpose(testPatterns))

  ### Computing the weights matrix W
  XX =  (inv(np.dot(featureValuesTrain, np.transpose(featureValuesTrain))))
  W  = np.dot(np.dot(XX,featureValuesTrain), np.transpose(trainLabels))
  W  = np.transpose(W)

  ### Computing the loss function with nrOfMisclassifications on TRAINING DATA
  classificationHypothesesTrain = np.dot(W,featureValuesTrain)

  maxValues, maxIndicesTrain = [], []
  maxValues_temp, maxIndicesTrain_temp = [], []
  for i in range(0, len(classificationHypothesesTrain[0])):
      index = np.argmax(np.asarray(classificationHypothesesTrain[:,i]))
      value = classificationHypothesesTrain[index][i]
      maxValues_temp.append(value)
      maxIndicesTrain_temp.append(index+1)

  maxValues.append(maxValues_temp)
  maxIndicesTrain.append(maxIndicesTrain_temp)
  
  trainMisclassificationRate = 100* nrOfMisclassificationsTrain(correctLabelsTrain,maxIndicesTrain) / len(classificationHypothesesTrain[0])

  ### Computing the loss function with nrOfMisclassifications on TEST DATA
  classificationHypothesesTest = np.dot(W,featureValuesTest)
  maxValuesTest, maxIndicesTest = [], []
  maxValuesTest_temp, maxIndicesTest_temp = [], []

  for i in range(0, len(classificationHypothesesTest[0])):
      index = np.argmax(np.asarray(classificationHypothesesTest[:,i]))
      value = classificationHypothesesTest[index][i]
      maxValuesTest_temp.append(value)
      maxIndicesTest_temp.append(index+1)

  maxValuesTest.append(maxValuesTest_temp)
  maxIndicesTest.append(maxIndicesTest_temp)

  testMisclassificationRate = 100 * nrOfMisclassificationsTrain(correctLabelsTest,maxIndicesTest) / len(classificationHypothesesTest[0])

  return W, trainMisclassificationRate, testMisclassificationRate

def bestmodel(W, mean_meanTrainImages, testPatternsFinal):
  ###################################################################################
  ### create a row vector of correct class labels (from 1 ... 10 for the 10 classes). 
  ### This vector is the not the same for training and testing.  
  ###  20 IS FOR THE TESTING
  ### Correct labels for test data
  correctLabelsTest, tmpTest = [], []
  for i in range(0, 10):
      for j in range(i*20, (1+i)*20):
          tmpTest.append(i+1)
  correctLabelsTest.append(tmpTest)
  ##################################################################################

  ### Defining the X matrix given by featureValuesTest
  featureValuesTest  = np.dot(np.transpose(mean_meanTrainImages), np.transpose(testPatternsFinal))

  ### Computing the loss function with nrOfMisclassifications on TEST DATA
  classificationHypothesesTest = np.dot(W,featureValuesTest)
  maxValuesTest, maxIndicesTest = [], []
  maxValuesTest_temp, maxIndicesTest_temp = [], []

  for i in range(0, len(classificationHypothesesTest[0])):
      index = np.argmax(np.asarray(classificationHypothesesTest[:,i]))
      value = classificationHypothesesTest[index][i]
      maxValuesTest_temp.append(value)
      maxIndicesTest_temp.append(index+1)

  maxValuesTest.append(maxValuesTest_temp)
  maxIndicesTest.append(maxIndicesTest_temp)

  testMisclassificationRate = 100 * nrOfMisclassificationsTrain(correctLabelsTest,maxIndicesTest) / len(classificationHypothesesTest[0])

  return testMisclassificationRate




## Reconstruction on ultimate final test data

In [5]:
array_Weights, array_train_error, array_test_error = [], [], []
mean_Weights = [ [ 0 for i in range(10) ] for j in range(10) ]          #### Matrix with mean weigths
mean_meanTrainImages = [ [ 0 for i in range(10) ] for j in range(240) ] #### Matrix with mean of the meanTrainImages

nIterations = 10

for i in range (nIterations):
  x, y, z = cross_validation_split(trainPatternsFinal)
  Weights, train_error, test_error = trainingFunction(x,y,z)
  print("iteration: ", i, "Weights: ", np.shape(Weights), "% train_error: ",train_error, "   % test_error: ",test_error)
  
  array_Weights.append(Weights)
  array_train_error.append(train_error)
  array_test_error.append(test_error)

  mean_Weights += Weights
  mean_meanTrainImages += np.array(z)

mean_Weights /= nIterations
mean_meanTrainImages /= nIterations

print("% mean train_error: ",np.mean(array_train_error), "   % mean test_error: ",np.mean(array_test_error))

ultimate_test_error = bestmodel(mean_Weights, mean_meanTrainImages, testPatternsFinal)

print("% ultimate_test_error: ",ultimate_test_error)


iteration:  0 Weights:  (10, 10) % train_error:  10.25    % test_error:  7.0
iteration:  1 Weights:  (10, 10) % train_error:  10.125    % test_error:  9.5
iteration:  2 Weights:  (10, 10) % train_error:  10.1875    % test_error:  9.5
iteration:  3 Weights:  (10, 10) % train_error:  10.625    % test_error:  9.0
iteration:  4 Weights:  (10, 10) % train_error:  9.5    % test_error:  14.0
iteration:  5 Weights:  (10, 10) % train_error:  9.9375    % test_error:  10.5
iteration:  6 Weights:  (10, 10) % train_error:  9.875    % test_error:  10.0
iteration:  7 Weights:  (10, 10) % train_error:  9.75    % test_error:  9.0
iteration:  8 Weights:  (10, 10) % train_error:  9.875    % test_error:  10.5
iteration:  9 Weights:  (10, 10) % train_error:  9.8125    % test_error:  9.0
% mean train_error:  9.99375    % mean test_error:  9.8
% ultimate_test_error:  7.5


In [6]:
array_Weights, array_train_error, array_test_error = [], [], []
mean_Weights = [ [ 0 for i in range(10) ] for j in range(10) ]          #### Matrix with mean weigths
mean_meanTrainImages = [ [ 0 for i in range(10) ] for j in range(240) ] #### Matrix with mean of the meanTrainImages

counts = 0
i_iteration = 0


while counts < 20:
  x, y, z = cross_validation_split(trainPatternsFinal)
  Weights, train_error, test_error = trainingFunction(x,y,z)

  i_iteration += 1

  if test_error < 6:
    print("iteration: ", i_iteration, "Weights: ", np.shape(Weights), "% train_error: ",train_error, "   % test_error: ",test_error)
    array_Weights.append(Weights)
    array_train_error.append(train_error)
    array_test_error.append(test_error)

    mean_Weights += Weights
    mean_meanTrainImages += np.array(z)

    counts += 1

mean_Weights /= nIterations
mean_meanTrainImages /= nIterations

print("% mean train_error: ",np.mean(array_train_error), "   % mean test_error: ",np.mean(array_test_error))

ultimate_test_error = bestmodel(mean_Weights, mean_meanTrainImages, testPatternsFinal)

print("% ultimate_test_error: ",ultimate_test_error)


iteration:  106 Weights:  (10, 10) % train_error:  10.1875    % test_error:  5.5
iteration:  388 Weights:  (10, 10) % train_error:  10.125    % test_error:  5.0
iteration:  410 Weights:  (10, 10) % train_error:  10.375    % test_error:  5.5
iteration:  589 Weights:  (10, 10) % train_error:  10.25    % test_error:  5.0
iteration:  716 Weights:  (10, 10) % train_error:  10.375    % test_error:  5.0
iteration:  1030 Weights:  (10, 10) % train_error:  10.125    % test_error:  4.5
iteration:  1235 Weights:  (10, 10) % train_error:  10.0625    % test_error:  5.0
iteration:  1313 Weights:  (10, 10) % train_error:  10.5625    % test_error:  5.0
iteration:  1345 Weights:  (10, 10) % train_error:  9.9375    % test_error:  5.5
iteration:  1567 Weights:  (10, 10) % train_error:  10.125    % test_error:  5.5
iteration:  1569 Weights:  (10, 10) % train_error:  10.4375    % test_error:  5.5
iteration:  1678 Weights:  (10, 10) % train_error:  9.9375    % test_error:  5.5
iteration:  1764 Weights:  (10